# imports

In [72]:
import pandas as pd
import sqlite3
import numpy as np
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity

In [67]:
def get_data_from_db(database_directory, tablename):
    db = sqlite3.connect(database_directory)
    query = f"SELECT * FROM {tablename}"
    df = pd.read_sql(query, db)
    return df

def create_users_vs_movies_matrix(df):
    df = df.drop("timestamp", axis = 1)
    df = df.set_index(["userId", "movieId"])
    df = df.unstack()
    df = df.fillna(0)
    return df

# def get_filterd_movie_ids(users_vs_movies_matrix):
# Paul's function here

In [132]:
d0 = get_data_from_db("data/movies.db", "ratings")

In [133]:
users_vs_movies_matrix = create_users_vs_movies_matrix(d0)
filtered_users_vs_movies_matrix = users_vs_movies_matrix.copy()
#filtered_users_vs_movies_matrix = users_vs_movies_matrix[filtered_columns]

# Collaborative Filtering

In [125]:
# Functions for comparing users against each other using cosine similarity
def make_cosine_heatmap(df):
    labels = list(df.index.values) 
    cosine_similarities = cosine_similarity(df)
    results_df = pd.DataFrame(cosine_similarities, columns = labels, index = labels)
    return results_df

def get_similar_users(cosine_heatmap, userId):
    results_for_one_user = cosine_heatmap.loc[userId].sort_values(ascending = False) # get similar users
    similar_users = list(results_for_one_user.keys())[1:]
    return similar_users

In [126]:
#users_vs_movies_matrix = create_users_vs_movies_matrix(d0)
cosine_heatmap = make_cosine_heatmap(users_vs_movies_matrix)
similar_users = get_similar_users(cosine_heatmap, 1)

In [139]:
#similar_users

In [138]:
recommended_movie_ids = ((filtered_users_vs_movies_matrix.loc[1] == 0.0) & (filtered_users_vs_movies_matrix.loc[266] >= 4.0))
#recommended_movie_ids
# make function that creates list of recommended movies
# --> FINISHED.

# NMF

In [135]:
def get_NMF_results_matrix(trained_NMF_model, users_vs_movies_matrix):
    R = users_vs_movies_matrix
    user_ids = list(R.index.values) 
    movie_ids = list(R.columns.get_level_values(1).values) 
    P = model.components_  # movie feature
    Q = model.transform(R)  # user feature
    nR = np.dot(Q, P)
    results = pd.DataFrame(nR, columns = movie_ids, index = user_ids)
    return results

In [136]:
R = users_vs_movies_matrix
model = NMF(n_components=2, init='random', random_state=10)
model.fit(R)

NMF(alpha=0.0, beta_loss='frobenius', init='random', l1_ratio=0.0,
  max_iter=200, n_components=2, random_state=10, shuffle=False,
  solver='cd', tol=0.0001, verbose=0)

In [115]:
NMF_results_matrix = get_NMF_results_matrix(model, R)

In [137]:
#NMF_results_matrix

In [ ]:
#query = [[0, 0, 5, 0]] 
#model.transform(query)
#apply_filter by Paul
# --> FINISHED.